In [13]:
### LLM
from langchain_ollama import ChatOllama

local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [14]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

## Call receptionist agent
receptionist_prompt = """
You are a receptionist at a law firm. A client has approached and the following is the problem they are facing.
Your job is to redirect them to the right specialised lawyer you who can help them. Return JSON with a single key, lawyer, that is one of the following options that the client is likely to find the best help from
[criminal, family, wills and estates]
"""

def receptionist_prompt_response(prompt):
  result = llm_json_mode.invoke(
    [SystemMessage(content=receptionist_prompt)]
    + [HumanMessage(content=prompt)]
  )
  return json.loads(result.content)

# print(
#   receptionist_prompt_response("I would like to divorce my husband"),
#   receptionist_prompt_response("I have been caught for trespassing"),
#   receptionist_prompt_response("I have some estate that I would like to inherit from my grandparents"),
#   )


In [15]:
## Context for wills agent

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

urls = [
  # "https://www.gov.uk/guidance/sign-in-to-your-hmrc-business-tax-account",
  # "https://lilianweng.github.io/posts/2023-06-23-agent/",
  # "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
  # "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
  "/Users/adiprabs/Coding/RAG/Wills_and_Estates.pdf",
]

relevant_docs = {
  "wills and estates" : ["/Users/adiprabs/Coding/RAG/Wills_and_Estates.pdf"],
  "criminal" : ["/Users/adiprabs/Coding/RAG/Wills and Estates.pdf"],
  "family" : ["/Users/adiprabs/Coding/RAG/Wills and Estates.pdf"],
}

# Load documents
def load_docs(category):
  docs = []
  for path in relevant_docs[category]:
    if path.startswith('http'):
      docs.append(WebBaseLoader(path).load())
    else:
      docs.append(PyPDFLoader(path).load())
  docs_list = [item for sublist in docs for item in sublist]


  # Split documents
  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
  )
  doc_splits = text_splitter.split_documents(docs_list)

  # Add to vectorDB
  vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
  )

  # Create retriever
  return vectorstore.as_retriever(k=3)

In [16]:
### Generate

# Prompt
rag_prompt = """
You are a lawyer tasked with getting all relevant information for a potential case. 
You need to ask relevant questions to get the information you need to help your client.
Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review your conversation history with the user

{history}

Ask a question that will help you get information you do not already have that may be essential for the case. Only repeat questions if you feel earlier answers have not given adequate information, while specifying what exactly you need. 
You will need the following details:

{details}

If you have no more questions, specifically say the phrase "Now over".

Question:"""


# Post-processing
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)


messages = ""

information_needed = {
  "wills and estates" : "full names, dates of birth, current marital status, address and contact information, aliases and other names, children (their names and dob), dependents, all assests owned jointly and individually, debts and liabilities associated with assets, any assests aborad, heirlooms, backup executor",
  "contract" : "full names, dates of birth, current marital status, address and contact information, aliases and other names, children (their names and dob), dependents, all assests owned jointly and individually, debts and liabilities associated with assets, any assests aborad, heirlooms, backup executor",
}

# Test
def lawyer_response(initial_query, category, retriever):
  docs = retriever.invoke(initial_query)
  docs_txt = format_docs(docs)
  messages = f"user: {initial_query}\n"
  for i in range(10):
    rag_prompt_formatted = rag_prompt.format(context=docs_txt, history=messages, details=information_needed[category])
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    messages += f"lawyer: {generation.content}\n"
    print(generation.content)
    if "Now over" in generation.content:
      break
    answer = input(">>>")
    print(answer)
    if answer == "/exit":
      break
    messages += f"user: {answer}\n"

  return (messages, docs_txt)

# wills_lawyer_response("I would like to inherit my grandparents' estate")

In [20]:
sample1 = """
Lawyer: Can you please introduce yourself and tell me why you're seeking 
legal advice?

Potential Client: My name is John Smith. I'm a local business owner and 
I've been having some issues with a contract dispute.

Lawyer: What kind of contract are we talking about? Is it a employment 
contract, a commercial contract, or something else?

Potential Client: It's a commercial contract for a service agreement.

Lawyer: And what is the nature of the dispute? Are you being sued or are 
you filing a lawsuit against someone?

Potential Client: I'm being sued by one of my business partners.

Lawyer: Okay. Can you tell me more about your relationship with this 
business partner? How long have you known them and what was your original 
agreement?

Potential Client: We've been business partners for about 5 years. Our 
original agreement was to share profits and losses equally.

Lawyer: And did you both agree on all the terms of the contract, or were 
there any disagreements?

Potential Client: There were some disagreements, but we always managed to 
work things out in the end.

Lawyer: Okay. What is your desired outcome in this case? Are you looking 
for a specific amount of money or a certain resolution to the dispute?

Potential Client: I just want to resolve the situation and move on with my 
business.
"""

sample2 = """
Lawyer: Can you please introduce yourself and tell me why you're seeking 
legal advice?

Potential Client: Hi, I'm Emily. I'm a small business owner and I've been 
experiencing some issues with one of my contractors.

Lawyer: What kind of contractor are we talking about? Is it a construction 
company or someone else?

Potential Client: It's a construction company that's been working on a 
project for me. But to be honest, I'm not really sure how it all works.

Lawyer: Okay, Emily. Can you tell me more about the project and what kind 
of issues you're experiencing? What makes you think you need legal advice?

Potential Client: Well, I've been having some problems with the quality of 
work and I'm worried that I might be liable for something. But I don't 
really know how it all works or what my rights are as a business owner.

Lawyer: Okay, Emily. Let's start with the basics. Can you tell me about 
the contract you have in place with this contractor? Do you have any 
written agreement or is everything oral?

Potential Client: Um, I think we had something in writing at first, but 
I'm not really sure what it says anymore. And even if I did remember, I'm 
not sure I can find it.

Lawyer: Okay, Emily. Can you try to think back and see if there were any 
specific incidents or issues that led up to this point? Was there a 
particular date or project milestone where things started to go wrong?

Potential Client: (thinks for a moment) Yeah, I remember one time when 
they didn't finish the job on time. But I don't know what my options are 
now.

Lawyer: Okay, Emily. Well, let's take this one step at a time. Can you 
tell me more about the contractor and their company? Who is the owner and 
how long have you been working with them?

Potential Client: (pauses) To be honest, I'm not really sure. They just 
showed up one day and started working on my project.

Lawyer: Okay, Emily. Well, this is a bit of a problem because it sounds 
like you may be dealing with an unlicensed contractor. Can you try to find 
out more information about their company and the owner?

Potential Client: (nods) Yeah, I can do that. But in the meantime, what 
can you help me with? Do you have any suggestions for how to move forward?

Lawyer: Well, Emily, I think we need to gather a bit more information 
before we can start advising you on how to proceed. But I can offer you 
some general guidance and next steps. Would that be okay with you?

In this example, the potential client is seeking help but does not have 
all the information they need. They are willing to learn and take action, 
but need guidance and direction from the lawyer to move forward. The 
conversation is still productive, despite the lack of complete 
information, and the lawyer is able to offer some general advice and next 
steps while also encouraging the client to gather more information.
"""

In [26]:
secretary_prompt = """
You are the tasked with compiling all relevant information from a potential client. Below is the conversation history between the client and the lawyer.
compile a report will all relevant information that the lawyer may need to help the client.

{history}

Present a concise report in with the following titles and their relevant information:
name, case summary, location, will, executor, beneficiaries, guardian, funeral arrangements, other wishes. At then end of the report, cite relevant laws or principles that may apply to this case from the following context.

{context}

If any information is missing or insufficient, specify missing for that tag.
"""

def secretary_response(msg_history, context):
  secretary_prompt_formatted = secretary_prompt.format(context=context, history=msg_history)
  result = llm.invoke(
    [HumanMessage(content=secretary_prompt_formatted)]
    )
  return result.content

example = """

"""

print(secretary_response(sample1, ""))


**Case Summary**

* Client Name: John Smith
* Case Type: Commercial Contract Dispute
* Nature of Dispute: Being sued by business partner over a service agreement
* Desired Outcome: Resolve situation and move on with business

**Location**
Missing information: Specific location of the business or dispute.

**Will**
Missing information: Whether John Smith has a will, if so, what are its contents?

**Executor**
Missing information: If John Smith has a will, who is designated as the executor?

**Beneficiaries**
Missing information: If John Smith has a will, who are the beneficiaries?

**Guardian**
Missing information: If John Smith has minor children or dependents, who would be their guardian?

**Funeral Arrangements**
Missing information: What are John Smith's wishes regarding funeral arrangements?

**Other Wishes**
Missing information: Are there any other specific wishes or instructions that John Smith wants to convey?

Relevant Laws or Principles:

* The Uniform Commercial Code (UCC) ma

In [24]:
summary = input("Tell us what you'd like to file a case for: ")
if summary == "exit":
  exit("exited")
lawyer = receptionist_prompt_response(summary)
retriever = load_docs(lawyer["lawyer"])
(conversation, context) = lawyer_response(summary, lawyer["lawyer"], retriever)
secretary_response(conversation, context)

ValueError: File path /Users/adiprabs/Coding/RAG/Wills and Estates.pdf is not a valid file or url